In [ ]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   #if like me you do not have a lot of memory in your GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "" #then these two lines force keras to use your CPU
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import h5py
import librosa
import os as os
import pandas as pd
import re
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score
from matplotlib import cm, colors, colorbar
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
rdg = RidgeClassifier(alpha=0.5)
#mlp=MLPClassifier(random_state=1,max_iter=300,activation='relu',solver='sgd',learning_rate='constant',learning_rate_init=0.0001)
mlp=MLPClassifier(random_state=1,max_iter=300,activation='relu')
from sklearn.linear_model import LogisticRegression
lgr=LogisticRegression(random_state=1,max_iter=500)
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=0,max_depth=10)
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(n_estimators=100, random_state=0)
from sklearn.ensemble import GradientBoostingClassifier
gbc= GradientBoostingClassifier(n_estimators=100, random_state=1)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
from sklearn.linear_model import SGDClassifier
#SGD=SGDClassifier(loss= 'log',random_state=1,max_iter=100,early_stopping=True,learning_rate='optimal',validation_fraction=0.2)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()
mmscaler= MinMaxScaler()
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10, random_state=0)
from sklearn.svm import SVC
clf_svm=SVC(kernel='rbf')
linear_svm=SVC(kernel='linear')
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from scipy.stats.mstats import mquantiles
from scipy.stats import skew
from sklearn.cluster import KMeans
from sklearn.model_selection import LeaveOneOut
pca = PCA(n_components=2, svd_solver='full')
import random
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split

In [ ]:
def read_eeg(data):
    with open(data) as f:
        raw = []
        for line in f:
            line = line.split() # to deal with blank 
            if line:            # lines (ie skip them)
                line = [int(float(i)) for i in line]
                raw.append(line)
    df = pd.DataFrame (raw,columns=['Channel_1','Channel_2','Channel_3','Channel_4','Channel_5','Channel_6','Channel_7','Channel_8',
                            'Channel_9','Channel_10','Channel_11','Channel_12','Channel_13','Channel_14','Channel_15'
                             ,'Channel_16','Channel_17','Channel_18','Channel_19'])
    return df

from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def noise_filter(df):
    filt_data=pd.Series()
    for i in range(0, df.shape[1]):
        signal = df.iloc[:,i].values
        data=butter_bandpass_filter(signal,0.1,45,128,3)
        
        filt_data=pd.concat([filt_data,pd.Series(data)],axis=1)
    filt_data=filt_data.iloc[:,1:]
    filt_data.columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19']
    return filt_data

In [ ]:
total_tps=34944-5*128 # cut all EEG of all subjects into equal length = 34944-5*128=34304 time points
number_of_epochs=26 # segment each subject's EEG into 26 epochs
number_of_tps=1280 # each epoch include 1280 time points

In [ ]:
os.chdir("/home/jupy/Raw_data/Time_0/Obese")
ob_0=os.listdir(os.getcwd())
ob_0.remove('.ipynb_checkpoints')
#ob_0.remove('New')

eeg_ob_0=np.zeros([len(ob_0),total_tps,19])
for i in range(0,len(ob_0)):
    eeg_ob_0[i,:,:]=(read_eeg(ob_0[i]).values)[0:total_tps,:]

os.chdir("/home/jupy/Raw_data/Time_0/Lean")
l_0=os.listdir(os.getcwd())
l_0.remove('.ipynb_checkpoints')
#l_0.remove('New')

eeg_l_0=np.zeros([len(l_0),total_tps,19])
for i in range(0,len(l_0)):
    eeg_l_0[i,:,:]=(read_eeg(l_0[i]).values)[0:total_tps,:]
    
X_0=np.vstack([eeg_ob_0,eeg_l_0])
print ('eeg_ob_0',eeg_ob_0.shape)
print ('eeg_l_0',eeg_l_0.shape)
print('X_0',X_0.shape)

In [ ]:
y=[0]*30+[1]*30

X_0_clean=np.zeros([60, 34304, 19])
for i in range(0,X_0.shape[0]):
    X_0_clean[i,:,:]=noise_filter(pd.DataFrame(X_0[i,:,:]))
X_0_clean=X_0_clean[:,0:int(number_of_epochs*number_of_tps),:]

X2_0=np.zeros([number_of_epochs*X_0_clean.shape[0],int(number_of_tps),19])
for chan in range(0,19):
    X1=np.zeros([number_of_epochs*X_0_clean.shape[0],int(number_of_tps)])
    for subj in range(0,X_0_clean.shape[0]):
        X1[subj*number_of_epochs:(subj+1)*number_of_epochs,:]=X_0_clean[subj,0:int(number_of_epochs*number_of_tps),chan].reshape(int(number_of_epochs),int(number_of_tps))
    X2_0[:,:,chan]=X1

In [ ]:
XX_0=np.zeros([X2_0.shape[0],19,number_of_tps])
for i in range(0,X2_0.shape[0]):
    XX_0[i,:,:]=X2_0[i,:,:].transpose()
print(XX_0.shape)

In [ ]:
Y2=[]
for n in y:
    Y2=Y2+[n]*number_of_epochs
len(Y2)

In [ ]:
del eeg_l_0
del eeg_ob_0
del X_0
del X_0_clean
del X2_0

In [ ]:
y_test_short=[1,1,1,0,0,0]

def subj_class_prob(result):
    pred=[]
    for i in range(0,6):
        #print(np.where(np.array(result[i*number_of_epochs:(i+1)*number_of_epochs])==1)[0].shape[0]/number_of_epochs)
        print(sum(predicted[i*number_of_epochs:(i+1)*number_of_epochs])[0], sum(predicted[i*number_of_epochs:(i+1)*number_of_epochs])[1])
        if sum(predicted[i*number_of_epochs:(i+1)*number_of_epochs])[0]>sum(predicted[i*number_of_epochs:(i+1)*number_of_epochs])[1]:
            pred.append(0)
        else:
            pred.append(1)
    return pred

# VAE

In [ ]:
from keras_self_attention import SeqSelfAttention
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout,Add,LSTM,Reshape,Bidirectional
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,AveragePooling1D,GlobalAveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Input, Conv1D,Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Dense, Lambda, Reshape, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Conv1D,Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Dense, Lambda, Reshape, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.preprocessing import minmax_scale

# feature extraction

In [ ]:
os.chdir('/home/jupy/SavedModel/autoencoder/T0_to_T0')
#os.chdir('/home/jupy/SavedModel/autoencoder/T15_to_T15')
mm_feature_output_0_0={}
for n in range(0,60):#2-48
    xar_0=XX_0[n*26:(n+1)*26,:,:]
    xar_norm_0=np.zeros([xar_0.shape[0],xar_0.shape[1],xar_0.shape[2]])
    for i in range(0,xar_0.shape[0]):
        for j in range(0,xar_0.shape[1]):
            xar_norm_0[i,j,:]=minmax_scale(xar_0[i,j,:])
    xar_0=xar_norm_0
    xar_0=np.expand_dims(xar_0,axis=-1)
    mm=load_model(f'T0_to_T0_VAE{n}')

    intermediate_layer_model = Model(inputs=mm.input,outputs=mm.get_layer('spatial_conv').output)
    #intermediate_layer_model.compile(optimizer=Adam(learning_rate=0.001))
    mm_feature_output_0_0[n]= intermediate_layer_model.predict(xar_0)
    
adapt_auto_feat_0_0=np.zeros([26*60,1,1217,8])
for n in range(0,60):
    adapt_auto_feat_0_0[n*26:(n+1)*26,:,:,:]=mm_feature_output_0_0[n]
Adapt_feat_0_0=np.zeros([26*60,1217*8])
for i in range(0,adapt_auto_feat_0_0.shape[0]):
    Adapt_feat_0_0[i,:]=adapt_auto_feat_0_0[i,:,:,:].flatten()

In [ ]:
Adapt_feat_0_0.shape

In [ ]:
raw_0=np.zeros([XX_0.shape[0],(XX_0.shape[1]*XX_0.shape[2])])
for i in range(0,1560):
    raw_0[i,:]=XX_0[i,:,:].flatten()

### Convex Hull


# Genetic Algorithm

0-125: good qulaity
126-220:middle quality
220-:bad quality

In [ ]:
subj_y=[]
for i in range(0,int(Adapt_feat_0_0.shape[0]/26)):
     subj_y.append([i]*26)
subj_y=np.array(subj_y).flatten()
subj_y.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


n_estimators = 200
rf = RandomForestClassifier(n_estimators=n_estimators, bootstrap=True)
X_train, X_test, y_train, y_test = train_test_split(Adapt_feat_0_0, quality_y, test_size=0.2, random_state=42,stratify=quality_y
)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

rf.feature_importances_
feat_id=rf.feature_importances_.argsort()[-500:][::-1]

# clustering

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2) 
ardata=Adapt_feat_0_0
tsardata=tsne.fit_transform(ardata)

In [ ]:
inertia = []
for k in range(1, 20):  # You can adjust the range of k as needed
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(tsardata)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(range(1, 20), inertia, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(range(1, 20))
#plt.title('Elbow for Optimal k')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
n_clusters = 5 # Number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(ardata)
cluster_assignments = kmeans.labels_

In [ ]:
cluster_assignments.shape
cluster_purity=[]
for i in range(0,60):
    cluster_count=(np.unique(cluster_assignments[i*26:(i+1)*26], return_counts=True)[0].shape[0])
    #print(np.unique(cluster_assignments[i*26:(i+1)*26], return_counts=True)[0].shape[0])
    cluster_purity.append(cluster_count)

In [ ]:
counts, bins, _ = plt.hist(cluster_purity, bins=9)
for count, x in zip(counts, bins):
    if count > 0:  # Remove 0 count
        plt.text(x + (bins[1] - bins[0]) / 2, count, str(int(count)), ha='center', va='bottom', fontsize=10, color='black')

plt.xlabel('Number of Clusters')
plt.ylabel('Number of Subjects')
plt.xticks(range(1, 6))  # Set the x-axis to start from 1 to 9
plt.yticks(range(0, 23, 5))  # Set the y-axis range from 0 to 22 with intervals of 5

plt.show()


In [ ]:

good_q_id=np.where(np.array(cluster_purity)<=2)[0]
midhigh_q_id = np.where((2 < np.array(cluster_purity)) & (np.array(cluster_purity) <=3))[0]
midlow_q_id = np.where((3< np.array(cluster_purity)) & (np.array(cluster_purity) <= 3.5))[0]
bad_q_id = np.where(np.array(cluster_purity)>3.5)[0]

print('good subj',good_q_id)
print('midhigh subj',midhigh_q_id)
print('midlow subj',midlow_q_id)
print('bad subj',bad_q_id)

quality_y=np.empty([1560,])
#quality_ar=np.zeros[1560,9736]
for i in good_q_id:
    #print(np.arange(i*26,(i+1)*26))
    quality_y[np.arange(i*26,(i+1)*26)]=0
    
for j in midhigh_q_id:
    quality_y[np.arange(j*26,(j+1)*26)]=1
    
for r in midlow_q_id:
    quality_y[np.arange(r*26,(r+1)*26)]=1
    
for z in bad_q_id:
    quality_y[np.arange(z*26,(z+1)*26)]=2

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Number of trees in the Random Forest
n_estimators = 200

# Create a Random Forest classifier with bootstrapping
rf = RandomForestClassifier(n_estimators=n_estimators, bootstrap=True)

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Adapt_feat_0_0, quality_y, test_size=0.2, random_state=42,stratify=quality_y
)#, 
# Train the Random Forest model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


In [ ]:
rf.feature_importances_
feat_id=rf.feature_importances_.argsort()[-500:][::-1]

# train/test split

Insert a token as subeject ID


In [ ]:
s_range=range(0,10)

In [ ]:
data_ind=[]
for n in range(0,10):
    data_ind.append(np.array([30,31,32,0,1,2])+n*3)

Xtest_id={}
Xtv_id={}


for s in s_range:    
    Xtest_id[s]=data_ind[s]
    #Xtv_id[s]=flatten(data_ind[0:s]+data_ind[(s+1):])
    Xtv_id[s]=(np.array(data_ind[0:s]).flatten().tolist())+(np.array(data_ind[(s+1):]).flatten().tolist())
    print('set',s,Xtest_id[s])
    print('set',s,Xtv_id[s])
    print('----------------------------')
    
Xdata= #raw_0#Adapt_feat_0_0#[:,feat_id]
Ydata=np.array(Y2)#quality_y#
Xtv={}
Ytv={}
Xtest={}
Ytest={}
for s in s_range:
    print('set',s)
    Xtest[s]=np.zeros([Xdata.shape[0],Xdata.shape[1]])
    Ytest[s]=np.zeros([Xdata.shape[0],])
    print('test_id',Xtest_id[s])
    for test_id in Xtest_id[s]:
            Xtest[s]=np.vstack([Xtest[s],Xdata[test_id*number_of_epochs:(test_id+1)*number_of_epochs,:]])
            Ytest[s]=np.hstack([Ytest[s],Ydata[test_id*number_of_epochs:(test_id+1)*number_of_epochs,]])
          
    Xtest[s]=Xtest[s][Xdata.shape[0]:,:] 
    Ytest[s]=Ytest[s][Xdata.shape[0]:,] 
    
    Xtv[s]=np.zeros([Xdata.shape[0],Xdata.shape[1]])
    Ytv[s]=np.zeros([Xdata.shape[0],])
    print('tv_id',Xtv_id[s])
    for tv_id in Xtv_id[s]:
            Xtv[s]=np.vstack([Xtv[s],Xdata[tv_id*number_of_epochs:(tv_id+1)*number_of_epochs,:]])
            Ytv[s]=np.hstack([Ytv[s],Ydata[tv_id*number_of_epochs:(tv_id+1)*number_of_epochs,]])
            
    Xtv[s]=Xtv[s][Xdata.shape[0]:,:] 
    Ytv[s]=Ytv[s][Xdata.shape[0]:,] 

In [ ]:
print(Xtv[s].shape, Ytv[s].shape)

In [ ]:
all_id_long={}
X_all_shuffled={}
Y_all_shuffled={}
Y_all_shuffled_cat={}
for s in s_range:
    np.random.seed(2)
    all_id_long[s]=np.arange(0,Xtv[s].shape[0])
    np.random.shuffle(all_id_long[s])
    
    X_all_shuffled[s]=np.zeros([ Xtv[s].shape[0], Xtv[s].shape[1]])
    Y_all_shuffled[s]=np.zeros([ Xtv[s].shape[0]])
    for n in range(0,all_id_long[s].shape[0]):
        X_all_shuffled[s][n,:]=Xtv[s][all_id_long[s][n],:]
        #print(n,all_id_long[s][n])
        Y_all_shuffled[s][n]=Ytv[s][all_id_long[s][n]]
    print(X_all_shuffled[s].shape)
    print(Y_all_shuffled[s].shape)

accuracy of subject filtered feautres vae

In [ ]:
scores=[]
for s in s_range:
    knn.fit(X_all_shuffled[s],Y_all_shuffled[s])
    print(accuracy_score(knn.predict(Xtest[s]),Ytest[s]))
    scores.append(accuracy_score(knn.predict(Xtest[s]),Ytest[s]))
print('avg',sum(scores)/len(scores))

accuracy of vae as features

In [ ]:
scores=[]
for s in s_range:
    knn.fit(X_all_shuffled[s],Y_all_shuffled[s])
    print(accuracy_score(knn.predict(Xtest[s]),Ytest[s]))
    scores.append(accuracy_score(knn.predict(Xtest[s]),Ytest[s]))
print('avg',sum(scores)/len(scores))

accuracy of raw eeg as features

In [ ]:
scores=[]
for s in s_range:
    knn.fit(X_all_shuffled[s],Y_all_shuffled[s])
    print(accuracy_score(knn.predict(Xtest[s]),Ytest[s]))
    scores.append(accuracy_score(knn.predict(Xtest[s]),Ytest[s]))
print('avg',sum(scores)/len(scores))

# 1D CNN

In [ ]:
def OneD_EEGNet(nb_classes=2,dropoutRate = 0.25,norm_rate = 0.25, dropoutType = 'Dropout'):

        input1   = Input(shape = (9736, 1))

        ##################################################################
        block1       = Conv1D(8, (64))(input1)
        block1       = BatchNormalization()(block1)
        block1       = Activation('relu')(block1)

        block1       = Conv1D(16, (32))(block1)
        block1       = BatchNormalization()(block1)
        block1       = Activation('relu')(block1)

        block1       = AveragePooling1D((4))(block1)
        block1       = Dropout(dropoutRate)(block1)

        block2       = Conv1D(32, (16))(block1)
        block2       = BatchNormalization()(block2)
        block2       = Activation('relu')(block2)

        block2       = AveragePooling1D((8))(block2)
        #block2       = tf.keras.layers.GlobalAveragePooling2D()(block2)
        block2       = Dropout(dropoutRate)(block2)

        flatten      = Flatten(name = 'flatten')(block2)
        initializer = initializer = tf.keras.initializers.Identity()
        dense        = Dense(nb_classes, name = 'dense', 
                             kernel_constraint = max_norm(norm_rate),kernel_initializer=initializer)(flatten)
        softmax      = Activation('softmax', name = 'softmax')(dense)

        return Model(inputs=input1, outputs=softmax)   

In [ ]:
 for s in range(0,10):
        
    lr=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.0005, decay_steps=50, decay_rate=0.9, staircase=False, name=None
    )
    adam=tf.keras.optimizers.Adam(learning_rate=0.001)#learning_rate=0.0005
    
    model=OneD_EEGNet()
    model.compile(loss = 'categorical_crossentropy', optimizer =adam,metrics=['accuracy']) # optimizer = 'adam'
    callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,restore_best_weights = True)
    history=model.fit( X_all_shuffled[s].reshape(1404, 9736,1),
                          to_categorical(Y_all_shuffled[s]),epochs=100, batch_size=100, validation_split=0.2, callbacks=[callback],verbose=1) 

In [ ]:
best_weights = model.get_weights()
model.set_weights(best_weights)

In [ ]:
quality_model=model

In [ ]:
#model.summary()

In [ ]:
#os.chdir('/home/jupy/SavedModel/autoencoder/T0_to_T0/classifier')
for s in range(0,10):
    #classifier=load_model(f'n_set{s}_btw_encoder_T0_classifier')
    classifier=model
    result=[]
    predicted=classifier.predict(Xtest[s])
    for i in range (0,np.array(predicted).shape[0]):
            if predicted[i][0]>predicted[i][1]:
                result.append([0])
            else:
                result.append([1])
    print('ep score',accuracy_score(result,Ytest[s]))
    pred=subj_class_prob(result)

    print('score',accuracy_score(pred,y_test_short))

In [ ]:
from sklearn.manifold import TSNE

tsne= TSNE(n_components=2, perplexity=15)
tsne_feat=tsne.fit_transform(Adapt_feat_0_0[:,feat_id])
tsne_vae=tsne.fit_transform(Adapt_feat_0_0)
tsne_org=tsne.fit_transform(raw_0)

In [ ]:

for i in [0, 10, 20, 29, 31, 40, 48, 59]:
    if i < 30:
        plt.scatter(
            tsne_org[i * 26:(i + 1) * 26, 0],
            tsne_org[i * 26:(i + 1) * 26, 1],
            alpha=0.7, marker='+',
            s=50, label='Lean')
    else:
        plt.scatter(
            tsne_org[i * 26:(i + 1) * 26, 0],
            tsne_org[i * 26:(i + 1) * 26, 1],
            alpha=0.5, marker='o',
            s=50, label='Obese')

#plt.legend(['Lean', 'Obese'])
plt.show()


In [ ]:
for i in [8, 4, 12, 24, 41, 49, 34, 50]:
    if i < 30:
        plt.scatter(
            tsne_vae[i * 26:(i + 1) * 26, 0],
            tsne_vae[i * 26:(i + 1) * 26, 1],
            alpha=0.7, marker='+',
            s=50, label='Lean')
    else:
        plt.scatter(
            tsne_vae[i * 26:(i + 1) * 26, 0],
            tsne_vae[i * 26:(i + 1) * 26, 1],
            alpha=0.5, marker='o',
            s=50, label='Obese')

#plt.legend(['Lean', 'Obese'])
plt.show()

In [ ]:


for i in [0, 10, 20, 29, 31, 40, 48, 59]:
    if i < 30:
        plt.scatter(
            tsne_feat[i * 26:(i + 1) * 26, 0],
            tsne_feat[i * 26:(i + 1) * 26, 1],
            alpha=0.7, marker='+',
            s=50, label='Lean')
    else:
        plt.scatter(
            tsne_feat[i * 26:(i + 1) * 26, 0],
            tsne_feat[i * 26:(i + 1) * 26, 1],
            alpha=0.5, marker='o',
            s=50, label='Obese')


# robust_feature_outputs.shape

In [ ]:
def OneD_EEGNet(nb_classes=2,dropoutRate = 0.25,norm_rate = 0.25, dropoutType = 'Dropout'):

        input1   = Input(shape = (9568, 1))

        ##################################################################
        block1       = Conv1D(8, (64))(input1)
        block1       = BatchNormalization()(block1)
        block1       = Activation('relu')(block1)

        block1       = Conv1D(16, (32))(block1)
        block1       = BatchNormalization()(block1)
        block1       = Activation('relu')(block1)

        block1       = AveragePooling1D((4))(block1)
        block1       = Dropout(dropoutRate)(block1)

        block2       = Conv1D(32, (16))(block1)
        block2       = BatchNormalization()(block2)
        block2       = Activation('relu')(block2)

        block2       = AveragePooling1D((8))(block2)
        #block2       = tf.keras.layers.GlobalAveragePooling2D()(block2)
        block2       = Dropout(dropoutRate)(block2)

        flatten      = Flatten(name = 'flatten')(block2)
        initializer = initializer = tf.keras.initializers.Identity()
        dense        = Dense(nb_classes, name = 'dense', 
                             kernel_constraint = max_norm(norm_rate),kernel_initializer=initializer)(flatten)
        softmax      = Activation('softmax', name = 'softmax')(dense)

        return Model(inputs=input1, outputs=softmax)   

In [ ]:
 for s in range(0,10):
        
    lr=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001, decay_steps=50, decay_rate=0.9, staircase=False, name=None
    )
    adam=tf.keras.optimizers.Adam(learning_rate=0.0001)#learning_rate=0.0005
    
    model=OneD_EEGNet()
    model.compile(loss = 'categorical_crossentropy', optimizer =adam,metrics=['accuracy']) # optimizer = 'adam'
    callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,restore_best_weights = True)
    history=model.fit( robust_feature_outputs.reshape(1404, 9568,1),
                          to_categorical(Y_all_shuffled[s]),epochs=100, batch_size=100, validation_split=0.2, callbacks=[callback],verbose=1) 

In [ ]:
best_weights = model.get_weights()
model.set_weights(best_weights)

In [ ]:
#robust_layer_model = Model(inputs=quality_model.input,outputs=quality_model.get_layer('average_pooling1d_1').output)
robust_feature_output_test= robust_layer_model.predict( Xtest[s])


robust_feature_outputs_test=np.zeros([Xtest[s].shape[0],int(299*32)])
for i in range(0,Xtest[s].shape[0]):
    robust_feature_outputs_test[i*26:(i+1)*26,:]=robust_feature_output_test[i,:,:].flatten()

In [ ]:
robust_feature_outputs_test.shape

In [ ]:
#model.summary()
Xtest[s].shape

In [ ]:
for s in range(0,10):
    #classifier=load_model(f'n_set{s}_btw_encoder_T0_classifier')
    classifier=model
    result=[]
    predicted=classifier.predict(robust_feature_outputs_test)
    for i in range (0,np.array(predicted).shape[0]):
            if predicted[i][0]>predicted[i][1]:
                result.append([0])
            else:
                result.append([1])
    print('ep score',accuracy_score(result,Ytest[s]))
    pred=subj_class_prob(result)

    print('score',accuracy_score(pred,y_test_short))